<h1>Convolutional Neural Network</h1>

In [ ]:
import numpy as np

from keras import backend as K
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils
import time

def get_time():
    return time.time()
start_time = get_time()

np.random.seed(20170430)

class LeNet:
    @staticmethod
    def build(input_shape, num_classes):
        print('input_shape', input_shape)
        print('num_classes', num_classes)
        model = Sequential()

        # first stage, conv => relu => pool
        model.add(Conv2D(20, kernel_size=5, padding="same",
                         input_shape=input_shape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # second stage, conv => relu => pool
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # third stage, flatten => relu
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # softmax
        model.add(Dense(num_classes))
        model.add(Activation("softmax"))

        return model


optimizer = Adam()

K.set_image_dim_ordering("th")

(X_train, y_train), (X_test, y_test) = mnist.load_data()

#X_train = X_train.reshape(60000, 784)
#X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

# 60K * [1 * 28 * 28]
X_train = X_train[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]

num_classes = 10

y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

input_shape = (1, 28, 28)
model = LeNet.build(input_shape, num_classes)
model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

tensorboard = TensorBoard(log_dir='./logs')
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=128,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[tensorboard])

score = model.evaluate(X_test, y_test, verbose=1)

print("\nTest score:", score[0])
print("Test accuracy:", score[1])

end_time = get_time()
last_time = end_time - start_time

print("Last time: " + str(last_time) + "s")